Importing the necessary libraries

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as condition
from selenium.webdriver.common.by import By
import time

Initiating all the variables

In [2]:
industry = 'Household Appliances'
country = 'China'
front_page = 'https://www.dnb.com/business-directory/company-information.medical_equipment_and_supplies_manufacturing.cn.html'
companies_in_a_page = 50
start_page = 19
end_page = 19
title_of_front_page = 'Medical Equipment And Supplies Manufacturing Companies In China'
maximum_wait_time = 20
excel_path = "c:\\Users\\Owner\\"  # need to double the \ from the original path
excel_name = country + '_' +  industry + str(start_page) + '-' + str(end_page)

Setting up the browser for web scraping

In [3]:
# I will be using Microsoft Edge as the browser
driver = webdriver.Edge()

# Opening the first page of the webpage
driver.get(front_page)

# Setting the waiting time 
wait = WebDriverWait(driver, maximum_wait_time)

# Waiting for the title of the front page to appear in the webpage
try: 
    wait.until(
        condition.text_to_be_present_in_element(
            (
                #XPATH for website title
                By.XPATH,
                ('//*[@id="page"]/div[2]/div/div[1]/div/div/div/div[2]/div/div/div/div'),
            ),
            title_of_front_page,
        ))

# Page will refresh after reaching the maximum wait time
except TimeoutException: 
    driver.refresh()

# There's always gonna be a cookie notification that will pop up
cookies_button = wait.until(
    condition.element_to_be_clickable((By.XPATH, '//*[@id="truste-consent-button"]')))

# so we will click it
cookies_button.click()


The start of the web scraping. We would get all the listed companies in each page together with their respective state, and directory link (will be used to get the full address and website)

In [4]:
# Lists to be used to append the extracted details
companies = []
broken_pages = []

# Looping over desired pages of the user
for page in range(start_page, end_page + 1):

    # Opening the page
    driver.get(front_page + f"?page={page}")

    # Waiting for the webpage to load
    try:
        wait.until(
            condition.text_to_be_present_in_element(
                (
                    By.XPATH,
                    (
                        # XPATH for the title of the page
                        '//*[@id="page"]/div[2]/div/div[1]/div/div/div/div[2]/div/div/div/div'
                    ),
                ),
                title_of_front_page,
            )
        )
    
    # There's instances that  the internet connection would be too slow and the challenge validation 
    # won't refresh after 10 secs, so this exception will take care of these cases
    except TimeoutException or 'Challenge Validation' in driver.title:
        driver.refresh()

    # Or rare instances that the page is just broken 
    # and we will take note of those
    except '505 Error' or 'Business Directory' in driver.title:
        broken_pages.append(front_page + f"?page={page}")
        continue

    # Extract the needed infos
    finally:

         # Gathering the necessary datas for each company by looping through each divs which starts at 2
        for i in range(2, companies_in_a_page + 2):

            # Getting the company name
            company = driver.find_element(
                By.XPATH, f'//*[@id="companyResults"]/div[{i}]/div[1]/a'
            )

            # Getting the diretory link 
            directory_link = driver.find_element(
                By.XPATH, f'//*[@id="companyResults"]/div[{i}]/div[1]/a'
            )

            # Getting the state of where the company is
            state = driver.find_element(
                By.XPATH, f'//*[@id="companyResults"]/div[{i}]/div[2]'
            )

            # Appending all those information in companies list
            companies.append(
                {
                    # address and website are left blank because we are only able fill out 
                    # after looping through the directory links
                    "company": company.text,
                    "directory_link": directory_link.get_attribute("href"),
                    "address": "",
                    "state": state.text.split(",")[0].replace("Country: ", ""),
                    "country": country,
                    "website": ""
                }
            )

Getting the address and website of the company by going through each of directory links 

In [5]:
# List of broken links 
broken_directory_links = []

# Looping through directory links to get the address and website
for company in companies:
    url = company["directory_link"]
    driver.get(url)

    # Waiting for the page to load
    try:
        wait.until(
            condition.element_to_be_clickable(
                # The discover span at the bottom of the page
                (By.XPATH, '//*[@id="footer_accordion_1_id"]/span[1]')
            )
        )

    # Refresh the page and wait for discover element at the bottom of the page
    # if the page took longer to load
    except TimeoutException:
        driver.refresh()
        wait.until(
            condition.element_to_be_clickable(
                (By.XPATH, '//*[@id="footer_accordion_1_id"]/span[1]')
            )
        )

    # If the title of the page is Business Directory then the company have no directory page
    except "Business Directory" in driver.title:
        continue

    # Getting the full address of the company 
    try:
        address = driver.find_element(
            By.XPATH, f'//*[@id="company_profile_snapshot"]/div[3]/div[2]/span/span'
        )
        # Updating the dictionary and removing the unnecessary text embedded in the element text
        company["address"] = address.text.replace(" See other locations", "")

    # There's rarely a case of a company that don't have address but we should catch that too
    except NoSuchElementException:
        pass
    
    # Getting the website of the company
    try:
        website = driver.find_element(
            By.XPATH, '//*[@id="company_new_header"]/div/div[2]/div[2]/div[1]/div[2]/a'
        )
        # Updating the dictionary
        company["website"] = website.get_attribute("href")

    # There's a lot of cases that there are no recorded sites
    except NoSuchElementException:
        pass


Taking a look at the scraped data

In [6]:
# Putting dictionary on DataFrame so we can export it to excel and have a better view
companies_df = pd.DataFrame(data = companies)

# We don't need the index
companies_df.reset_index(drop=True, inplace=True)

# First 5 rows will suffice
companies_df.head(5)

,company,directory_link,address,state,country,website
0,"Beier Taike Medical Instruments Jiangsu Co., Ltd.",https://www.dnb.com/business-directory/company...,"Jiangsu Yuma Aluminum Industry Co., Ltd., Erch...",Taizhou,China,http://www.better-tec.com/
1,"Foshan Dihua Technology Co., Ltd.",https://www.dnb.com/business-directory/company...,"1F, No.98, South Area, Guojiahuojujihua Foshan...",Foshan,China,http://www.dihua-tech.com/
2,"Shenzhen Chuanshi Biological Medical Co., Ltd.",https://www.dnb.com/business-directory/company...,"Floor 4, Building B, Haikexing Zhanlue Xinxing...",Shenzhen,China,http://www.thistory.com.cn/
3,"Kunshan Ideaman Intelligent Technology Co., Ltd.",https://www.dnb.com/business-directory/company...,"No.591, Shipai Liji Road, Bacheng Town Kunshan...",Kunshan,China,http://www.ksideaman.com/
4,"Cixi Bulade Medical Instruments Co., Ltd.",https://www.dnb.com/business-directory/company...,"No.60, Kandun W. Street, Kandun Sub-District N...",Ningbo,China,


Adding a column because we need this as input to advance search in DnB Finance Analytics, the platform which we will get the number of phone number, line of business, number of employees, and annual sales of the company.

In [7]:
companies_df["first3_address"] = companies_df["address"].apply(
    lambda x: (("Pass" if len(" ") >= 3 else " ".join(x.split(" ")[:3])))
)
companies_df.head(5)

,company,directory_link,address,state,country,website,first3_address
0,"Beier Taike Medical Instruments Jiangsu Co., Ltd.",https://www.dnb.com/business-directory/company...,"Jiangsu Yuma Aluminum Industry Co., Ltd., Erch...",Taizhou,China,http://www.better-tec.com/,Jiangsu Yuma Aluminum
1,"Foshan Dihua Technology Co., Ltd.",https://www.dnb.com/business-directory/company...,"1F, No.98, South Area, Guojiahuojujihua Foshan...",Foshan,China,http://www.dihua-tech.com/,"1F, No.98, South"
2,"Shenzhen Chuanshi Biological Medical Co., Ltd.",https://www.dnb.com/business-directory/company...,"Floor 4, Building B, Haikexing Zhanlue Xinxing...",Shenzhen,China,http://www.thistory.com.cn/,"Floor 4, Building"
3,"Kunshan Ideaman Intelligent Technology Co., Ltd.",https://www.dnb.com/business-directory/company...,"No.591, Shipai Liji Road, Bacheng Town Kunshan...",Kunshan,China,http://www.ksideaman.com/,"No.591, Shipai Liji"
4,"Cixi Bulade Medical Instruments Co., Ltd.",https://www.dnb.com/business-directory/company...,"No.60, Kandun W. Street, Kandun Sub-District N...",Ningbo,China,,"No.60, Kandun W."


We would also include record these broken link in Excel

In [8]:
# Putting dictionary on DataFrame so we can export it to excel and have a better view
broken_df = pd.DataFrame(
    {
        "Broken Pages in Directory": broken_pages,
        "Broken Links in Directory": broken_directory_links,
    }
)
# We don't need the index
broken_df.reset_index(drop=True, inplace=True)

# First 5 rows will suffice
broken_df.head(5)

,Broken Pages in Directory,Broken Links in Directory


Exporting the Data frames to Excel. The resulting Excel file will have 2 sheets : Scrape Data and Broken (Pages and Links)

In [9]:
# Creating an Excel file to write the results to
output_file = excel_path + excel_name + ".xlsx"

# Writing the Data Frames to Excel
writer = pd.ExcelWriter(output_file, engine= 'openpyxl')

# First sheet is alloted for the Scraped Data
companies_df.to_excel(writer,sheet_name = 'Scraped Data')

# The second sheet if for the list of broken pages and links
broken_df.to_excel(writer,sheet_name = 'Broken')

# Saving  and Closing the sheet
writer.close()